# Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import os
import sys
import pyaudio
import wave
import malaya_speech
import numpy as np
import librosa
from malaya_speech import Pipeline
import matplotlib.pyplot as plt
import IPython.display as ipd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import math 

# Silence Removal

In [ ]:
def silence_removal():
    y, sr = malaya_speech.load('/home/faaira/Desktop/fyp/output.wav')
    y_ = librosa.effects.trim(y, top_db = 20)[0]
    y_int = malaya_speech.astype.float_to_int(y)
    audio = AudioSegment(
        y_int.tobytes(),
        frame_rate = sr,
        sample_width = y_int.dtype.itemsize,
        channels = 1
    )
    audio_chunks = split_on_silence(
        audio,
        min_silence_len = 200,
        silence_thresh = -30,
        keep_silence = 100,
    )
    y1_ = sum(audio_chunks)
    y1_ = np.array(y1_.get_array_of_samples())
    y1_ = malaya_speech.astype.int_to_float(y1_)
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=("","Original Cough","", "After 1st Silence Removal","","After 2nd Silence Removal"))
    fig.add_trace(
        go.Scatter(y=y),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(y=y_),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(y=y1_),
        row=2, col=3
    )
    fig.update_layout(height=600, width=1200, title_text="Cough Detection Algorithm")
    pyo.plot(fig)

# Cough Detection

In [ ]:
def cough_detect():
    file = "/home/faaira/Desktop/fyp/output.wav"
    x,fs = librosa.load(file, sr=None)
    def segment_cough(x,fs, cough_padding=0.9,min_cough_len=0.2, th_l_multiplier = 0.1, th_h_multiplier = 2):
        cough_mask = np.array([False]*len(x))


        #Define hysteresis thresholds
        rms = np.sqrt(np.mean(np.square(x)))
        seg_th_l = th_l_multiplier * rms
        seg_th_h =  th_h_multiplier*rms

        #Segment coughs
        coughSegments = []
        padding = round(fs*cough_padding)
        min_cough_samples = round(fs*min_cough_len)
        cough_start = 0
        cough_end = 0
        cough_in_progress = False
        tolerance = round(0.01*fs)
        below_th_counter = 0

        for i, sample in enumerate(x**2):
            if cough_in_progress:
                if sample<seg_th_l:
                    below_th_counter += 1
                    if below_th_counter > tolerance:
                        cough_end = i+padding if (i+padding < len(x)) else len(x)-1
                        cough_in_progress = False
                        if (cough_end+1-cough_start-2*padding>min_cough_samples):
                            coughSegments.append(x[cough_start:cough_end+1])
                            cough_mask[cough_start:cough_end+1] = True
                elif i == (len(x)-1):
                    cough_end=i
                    cough_in_progress = False
                    if (cough_end+1-cough_start-2*padding>min_cough_samples):
                        coughSegments.append(x[cough_start:cough_end+1])
                else:
                    below_th_counter = 0
            else:
                if sample>seg_th_h:
                    cough_start = i-padding if (i-padding >=0) else 0
                    cough_in_progress = True

        return coughSegments, cough_mask

    cough_segments, cough_mask = segment_cough(np.asarray(x),fs)  
    fig = make_subplots(
        rows=len(cough_segments), cols=len(cough_segments),
        subplot_titles=("","Original Cough","" ,"","Cough Segments"))
    j=2
    k=1
    fig.add_trace(
        go.Scatter(y=x),
        row=1, col=math.ceil(len(cough_segments)/2)
    )
    for i in range(0,len(cough_segments)):
        if j == len(cough_segments):
            j+=1
        fig.add_trace(
        go.Scatter(y=cough_segments[i]),
        row=j, col=k
        )
        k+=1
    fig.update_layout(height=800, width=1200, title_text="Cough Detection Algorithm")
    pyo.plot(fig)
    silence_removal()

# Recording cough

In [ ]:
def record_audio():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    RECORD_SECONDS = 10
    WAVE_OUTPUT_FILENAME = "output.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    cough_detect()


In [ ]:
record_audio()

In [2]:
import webbrowser
chrome_path = '/usr/bin/google-chrome %s'
webbrowser.get(chrome_path).open('file:///home/faaira/Desktop/info project/Unsuccessful.html')

True